In [6]:
!pip install datasets
!pip install transformers
!pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 MB 2.0 MB/s eta 0:00:000m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.0 MB/s eta 0:00:00m eta 0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:000m eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.6 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.4 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:000m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.2 MB/s eta 0:00:000m eta 0:00:010:00:01

In [36]:
from datasets import load_dataset
ds = load_dataset("wmt/wmt14", "cs-en")

In [37]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

/home/tazik/mambaforge/envs/aj/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [42]:
import torch
import math

num_layers=6
seq_len = 1024 #TODO: this should be input dependent
d_model = 512

class FFN:
    def __init__(self, x):
        assert x.shape == (seq_len, d_model)
        
        self.d_ff = d_model*4
    
        self.W_1 = torch.nn.Linear(d_model, self.d_ff)
        self.b_1 = torch.zeros(seq_len, self.d_ff)
        
    def fwd(self):
        #TODO: max(0, ...)
        out1 = self.W_1(x) + self.b_1

        self.W_2 = torch.nn.Linear(self.d_ff, d_model)
        self.b_2 = torch.zeros(seq_len, d_model)
        out2 = self.W_2(out1) + self.b_2

        return out2
        # print(out2.shape)     

class MHA:
    def __init__(self, x_k, x_v, h=8, has_mask=False):
        assert x_k.shape == (seq_len, d_model)
        assert x_v.shape == (seq_len, d_model)
        
        self.has_mask = has_mask
        self.d_k = 8
        # self.d_k = d_model // h
        h_d_v = 64 
        #h_d_v = h*self.d_v
        
        self.d_v = self.d_k
        #self.scale = 1/math.sqrt(self.d_k)
        self.scale = 1

        assert d_model == 512
        assert self.d_k == 8

        self.x_k = x_k
        self.x_v = x_v
 
        self.W_Q = torch.zeros(size=(d_model, self.d_k))
        self.W_K = torch.zeros(size=(d_model, self.d_k))
        self.W_V = torch.zeros(size=(d_model, self.d_v))
        self.W_O = torch.zeros(size=(h_d_v, d_model))   
                

    def attn(self):

        Q = self.x_k @ self.W_Q
        K = self.x_k @ self.W_K
        V = self.x_v @ self.W_V

        #TODO: fix masking logic
        if self.has_mask:
            mask = torch.ones(seq_len, d_model) 
            last_idx = len(x)-1
            mask[last_idx:, :] = -float('inf')
            self.x_k = self.x_k * mask
            self.x_v = self.x_v * mask

        #TODO: dim=0 => softmax along seq_len
        
        head = torch.softmax(input=self.scale*(Q@K.T), dim=0) @ V
        return head

    def multi(self):
        heads = torch.cat([self.attn() for i in range(8)], dim=1)
        res = heads @ self.W_O
        return res   


class EncLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()          

    def forward(self, x):
        M = MHA(x_k=x, x_v=x)
        mha = M.multi()

        #sl1 = torch.nn.LayerNorm(x + mha)
        sl1 = x + mha        
        # print(sl1.shape)

        F = FFN(sl1)
        ffn = F.fwd()
        
        #sl2 = torch.nn.LayerNorm(sl1 + ffn)
        sl2 = sl1 + ffn
        # print(sl2.shape)
        
        return sl2

class DecLayer(torch.nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        #TODO: add LayerNorm after each sl{i}
        M1 = MHA(has_mask=True, x_k=x, x_v=x)
        mmha = M1.multi()

        sl1 = x + mmha

        M2 = MHA(has_mask=False, x_k=x, x_v=sl1)
        mha = M2.multi()

        sl2 = sl1 + mha

        F = FFN(sl2)
        ffn = F.fwd()

        sl3 = sl2 + ffn

        return sl3

class EncoderDecoder(torch.nn.Module):
    def __init__(self, x):
        assert x.shape == (seq_len, d_model)
        super().__init__()        
        self.x = x
        self.encs = [None] * num_layers
        self.decs = [None] * num_layers

    def forward(self):
        for i in range(num_layers):
            if i == 0:
                self.encs[i] = EncLayer()(self.x)
            else:
                self.encs[i] = EncLayer()(self.encs[i-1])
                
            self.decs[i] = DecLayer()(self.encs[i])

        return self.decs[num_layers-1]

class Transformer():
    def __init__(self, x):
       self.x = x 
       self.enc_dec = EncoderDecoder(self.x)()          

    def fwd(self):
       #TODO: add linear layer
       return torch.softmax(input=self.enc_dec, dim=0) #TODO: once again this is along seq_len
        

In [44]:
x = torch.rand(size=(seq_len, d_model))
trans = Transformer(x)
res = trans.fwd()
res.shape

torch.Size([1024, 512])